# Hermetic Wiper

Sample: `1bc44eef75779e3ca1eefb8ff5a64807dbc942b1e4a2672d77b9f6928d292591`



## References
- [Sample on Malshare](https://malshare.com/sample.php?action=detail&hash=1bc44eef75779e3ca1eefb8ff5a64807dbc942b1e4a2672d77b9f6928d292591)
- [J. A. Guerrero-Saade Twitter Thread](https://twitter.com/juanandres_gs/status/1496581710368358400?s=20&t=tuoQe510ow0HmQNbEXQimQ)
- [ESET Twitter Thread](https://twitter.com/ESETresearch/status/1496581903205511181?s=20&t=tuoQe510ow0HmQNbEXQimQ)
- [CrowdStrike Blog](https://www.crowdstrike.com/blog/how-crowdstrike-falcon-protects-against-wiper-malware-used-in-ukraine-attacks/)
- [SentinalOne Blog](https://www.sentinelone.com/labs/hermetic-wiper-ukraine-under-attack/)

## Backstory
- Wiper binary is signed using a code signing certificate issued to Wiper binary is signed using a code signing certificate issued to **Hermetica Digital Ltd** 
- Wiper abuses legitimate drivers from the **EaseUS Partition Master**
- Modifying CrashControl regkey, [CrashDumpEnabled](https://docs.microsoft.com/en-us/windows/client-management/system-failure-recovery-options#under-write-debugging-information) key to 0
- Enumerating PhysicalDrives up to 100 - can you even have 100 drives??

## Abused Legit Drivers
The following legit drivers are stored in the resources section of the PE. They are compressed.
- RCDATA_DRV_X64 (mscompress) e5f3ef69a534260e899a36cec459440dc572388defd8f1d98760d31c700f42d5
- RCDATA_DRV_X86 (mscompress) b01e0c6ac0b8bcde145ab7b68cf246deea9402fa7ea3aede7105f7051fe240c1
- RCDATA_DRV_XP_X64 (mscompress) b6f2e008967c5527337448d768f2332d14b92de22a1279fd4d91000bb3d4a0fd
- RCDATA_DRV_XP_X86 (mscompress) fd7eacc2f87aceac865b0aa97a50503d44b799f27737e009f91f3c281233c17d


## Reversing Notes

Arg1 is a sleep before reboot (in minutes)
Arg2 is sleep before ??? (minutes minus the arg1 sleep)

Grants itself `SeBackupPrivilege`, and `SeShutdownPrivilege`. [Note](https://docs.microsoft.com/en-us/windows-hardware/drivers/ifs/privileges), `SeBackupPrivilege` allows file content retrieval, even if the security descriptor on the file might not grant such access. 


If the host is Windows Vista and above they use one set of drivers (64/32bit). For XP they use a different set of drivers (32/64bit).

Dissable crashdump using reg key `SYSTEM\\CurrentControlSet\\Control\\CrashControl\\CrashDumpEnabled`

Possible driver communication via named pipe `\\.\EPMNTDRV\0`

Write the driver from resource to `\??\c:\Windows\system32\Drivers` as an mscompress file. Then copy and decompress it to another file with the same name and a `.sys` file extension. Then grant `SeLoadDriverPrivilege` priviledges to self and load driver.


